<a href="https://colab.research.google.com/github/WSmithDR/A-B-testing-project/blob/main/Regression_AB_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pruebas A/B de Marketing

Campaña de marketing simple con grupo experimental y grupo de control para pruebas A/B [(Dataset de Kaggle)](https://www.kaggle.com/datasets/faviovaz/marketing-ab-testing/data).

La mayoría de las personas estarán expuestas a anuncios (el grupo experimental). Y una pequeña porción de personas (el grupo de control) verían en su lugar un Anuncio de Servicio Público (PSA) (o nada) en el tamaño y lugar exactos donde normalmente estaría el anuncio.

**Diccionario de datos:**

- user id: ID de usuario (único)
- test group: Si es "ad", la persona vio el anuncio, si es "psa", solo vio el anuncio de servicio público
- converted: Si una persona compró el producto, entonces es True, de lo contrario es False
- total ads: Cantidad de anuncios vistos por la persona
- most ads day: Día en que la persona vio la mayor cantidad de anuncios
- most ads hour: Hora del día en que la persona vio la mayor cantidad de anuncios

In [1]:
%pip install numpy pandas statsmodels

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

## Check the data

In [3]:
# Read data and clean up column names
path_data = 'https://github.com/pabloestradac/experimentation-notebooks/raw/main/data/'
df = pd.read_csv(path_data + 'marketing_campaign.csv', index_col=0)
df = df.rename(columns=lambda x: x.strip().replace(" ", "_"))
df['treatment'] = np.where(df['test_group'] == 'ad', 1, 0)
df['converted'] = df['converted'].astype(int)
df.head()

,user_id,test_group,converted,total_ads,most_ads_day,most_ads_hour,treatment
0,1069124,ad,0,130,Monday,20,1
1,1119715,ad,0,93,Tuesday,22,1
2,1144181,ad,0,21,Tuesday,18,1
3,1435133,ad,0,355,Tuesday,10,1
4,1015700,ad,0,276,Friday,14,1


## Regression for A/B Testing

Si la campaña fue exitosa, ¿cuánto de ese éxito podría atribuirse a los anuncios?

1. Corre la siguiente regresion, e imprime los resultados.

$$
\text{converted} = \beta_0 + \beta_1 \text{treatment} + \epsilon
$$

2. ¿Que puedes concluir de los resultados de la regresion?

3. ¿Como se comparan estos resultados con el test de diferencia de medias e intervalos de confianza que obtuvimos en el primer notebook de A/B Test?

In [4]:
model1 = smf.ols('converted ~ treatment', data=df).fit()

In [5]:
display(model1.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              converted   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     54.32
Date:                Sun, 04 May 2025   Prob (F-statistic):           1.70e-13
Time:                        10:31:46   Log-Likelihood:             2.5499e+05
No. Observations:              588101   AIC:                        -5.100e+05
Df Residuals:                  588099   BIC:                        -5.099e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0179      0.001     17.459      0.000       0.016       0.020
treatment      0.0077      0.001      7.370      0.000       0.006       0.010
==============================================================================
Omnibus:                   650946.201   Durbin-Watson:                   1.975
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         32995692.936
Skew:                           6.053   Prob(JB):                         0.00
Kurtosis:                      37.641   Cond. No.                         9.90
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### ¿Cuánto de ese éxito podría atribuirse a los anuncios?

El coeficiente de treatment en la regresión es 0.0077. Esto significa que, en promedio, estar en el grupo de tratamiento: **ver el anuncio** aumenta la probabilidad de conversión en 0.77 puntos porcentuales respecto al grupo de control. Este valor representa el efecto atribuible a los anuncios.

### ¿Qué puedes concluir de los resultados de la regresión?

El efecto de los anuncios es positivo y estadísticamente significativo (p < 0.001). El valor del coeficiente indica que los anuncios tienen un impacto real, aunque pequeño, en la probabilidad de conversión.
El intercepto (0.0179) indica la tasa de conversión base del grupo de control (1.79%).
El R² es muy bajo (0.000), lo que es común en modelos de conversión binaria, pero no invalida la significancia del efecto estimado.

### ¿Cómo se comparan estos resultados con el test de diferencia de medias e intervalos de confianza que obtuvimos en el primer notebook de A/B Test?

Los resultados de la regresión son idénticos a los obtenidos con el test de diferencia de medias en el primer notebook:
El coeficiente de treatment en la regresión (0.0077) es exactamente igual a la diferencia de tasas de conversión entre los grupos experimental y de control (0.0255 - 0.0179 = 0.0076 ≈ 0.0077).
El valor p es muy bajo (p < 0.001) en ambos métodos, indicando que la diferencia es estadísticamente significativa.
El intervalo de confianza calculado en el primer notebook (0.0060 a 0.0094) contiene el valor estimado por la regresión y muestra que el efecto es positivo y significativo.
La regresión y el test de diferencia de medias llegan a la misma conclusión: los anuncios aumentan la tasa de conversión en aproximadamente 0.8 puntos porcentuales.
Esto ocurre porque, en este caso, la regresión simple y el test de diferencia de medias son matemáticamente equivalentes cuando la variable de tratamiento es binaria y la variable de resultado es también binaria.

## Additive Regression Model

1. Ahora corre el siguiente modelo de regresion aditivo. Asume que las covariables fueron recolectadas antes del experimento.

$$
\text{converted} = \beta_0 + \beta_1 \text{treatment} + \beta_2 \text{total ads} + \beta_3 \text{most ads day} + \beta_4 \text{most ads hour} + \epsilon
$$

2. ¿Como cambian los resultados anteriores?

In [6]:
model2 = smf.ols('converted ~ treatment + total_ads + C(most_ads_day) + most_ads_hour', data=df).fit()

In [7]:
display(model2.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              converted   R-squared:                       0.049
Model:                            OLS   Adj. R-squared:                  0.049
Method:                 Least Squares   F-statistic:                     3339.
Date:                Sun, 04 May 2025   Prob (F-statistic):               0.00
Time:                        10:31:49   Log-Likelihood:             2.6961e+05
No. Observations:              588101   AIC:                        -5.392e+05
Df Residuals:                  588091   BIC:                        -5.391e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       -0.0165      0.001    -13.154      0.000      -0.019      -0.014
C(most_ads_day)[T.Monday]        0.0117      0.001     16.144      0.000       0.010       0.013
C(most_ads_day)[T.Saturday]     -0.0001      0.001     -0.175      0.861      -0.002       0.001
C(most_ads_day)[T.Sunday]        0.0042      0.001      5.755      0.000       0.003       0.006
C(most_ads_day)[T.Thursday]      0.0020      0.001      2.770      0.006       0.001       0.003
C(most_ads_day)[T.Tuesday]       0.0102      0.001     13.660      0.000       0.009       0.012
C(most_ads_day)[T.Wednesday]     0.0047      0.001      6.333      0.000       0.003       0.006
treatment                        0.0075      0.001      7.348      0.000       0.005       0.009
total_ads                        0.0008   4.57e-06    171.208      0.000       0.001       0.001
most_ads_hour                    0.0007   4.13e-05     17.769      0.000       0.001       0.001
==============================================================================
Omnibus:                   620785.367   Durbin-Watson:                   1.993
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         28559140.446
Skew:                           5.611   Prob(JB):                         0.00
Kurtosis:                      35.242   Cond. No.                         415.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Interpretación del modelo aditivo

Esto significa que, controlando por el número total de anuncios, el día y la hora con más anuncios, estar en el grupo de tratamiento (ver el anuncio) aumenta la probabilidad de conversión en 0.75 puntos porcentuales respecto al grupo de control.
El efecto sigue siendo positivo y estadísticamente significativo (p < 0.001).
Otros coeficientes:
total_ads y most_ads_hour también tienen efectos positivos y significativos sobre la probabilidad de conversión.
Algunos días de la semana (most_ads_day) también muestran diferencias significativas respecto al día de referencia.
Intercepto: -0.0165
No tiene interpretación directa, ya que depende de la codificación de las variables categóricas.
R²: 0.049
El modelo ahora explica un 4.9% de la variabilidad en la conversión, lo que es una mejora respecto al modelo anterior (que tenía R² ≈ 0).

### ¿Cómo cambian los resultados anteriores?

Al incluir las variables (total_ads, most_ads_day, most_ads_hour), el coeficiente de treatment disminuye ligeramente de 0.0077 a 0.0075, pero sigue siendo positivo y altamente significativo.
Esto indica que el efecto de los anuncios sobre la conversión se mantiene incluso después de controlar por otras variables que podrían influir en la probabilidad de conversión. Además, el modelo aditivo explica una mayor proporción de la variabilidad en la conversión (R² = 0.049), lo que sugiere que las variables incluidas aportan información relevante para predecir la conversión. En resumen, los resultados son consistentes con el modelo simple: los anuncios tienen un efecto positivo y significativo, y este efecto es robusto al controlar por otras características de los usuarios.

## Interactive Regression Model

1. Ahora corre el siguiente modelo de regresion interactivo.

$$
\text{converted} = \beta_0 + \beta_1 \text{treatment} + \beta_2 \text{treatment} \times \text{total ads} + \beta_3 \text{treatment} \times \text{most ads day} + \beta_4 \text{treatment} \times \text{most ads hour} + \epsilon
$$

2. ¿Existen mejoras en la precision con respecto a los resultados del modelo aditivo?

In [8]:
model3 = smf.ols('converted ~ treatment * (total_ads + C(most_ads_day) + most_ads_hour)', data=df).fit()

In [9]:
display(model3.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              converted   R-squared:                       0.049
Model:                            OLS   Adj. R-squared:                  0.049
Method:                 Least Squares   F-statistic:                     1773.
Date:                Sun, 04 May 2025   Prob (F-statistic):               0.00
Time:                        10:31:52   Log-Likelihood:             2.6966e+05
No. Observations:              588101   AIC:                        -5.393e+05
Df Residuals:                  588083   BIC:                        -5.391e+05
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
==========================================================================================================
                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
Intercept                                 -0.0106      0.004     -2.673      0.008      -0.018      -0.003
C(most_ads_day)[T.Monday]                  0.0067      0.004      1.883      0.060      -0.000       0.014
C(most_ads_day)[T.Saturday]               -0.0042      0.004     -1.104      0.270      -0.012       0.003
C(most_ads_day)[T.Sunday]                  0.0038      0.004      1.029      0.303      -0.003       0.011
C(most_ads_day)[T.Thursday]                0.0029      0.003      0.823      0.410      -0.004       0.010
C(most_ads_day)[T.Tuesday]                 0.0007      0.004      0.187      0.852      -0.007       0.008
C(most_ads_day)[T.Wednesday]              -0.0001      0.004     -0.040      0.968      -0.007       0.007
treatment                                  0.0013      0.004      0.317      0.751      -0.007       0.009
treatment:C(most_ads_day)[T.Monday]        0.0051      0.004      1.400      0.162      -0.002       0.012
treatment:C(most_ads_day)[T.Saturday]      0.0043      0.004      1.102      0.270      -0.003       0.012
treatment:C(most_ads_day)[T.Sunday]        0.0004      0.004      0.106      0.916      -0.007       0.008
treatment:C(most_ads_day)[T.Thursday]     -0.0009      0.004     -0.241      0.809      -0.008       0.006
treatment:C(most_ads_day)[T.Tuesday]       0.0099      0.004      2.561      0.010       0.002       0.017
treatment:C(most_ads_day)[T.Wednesday]     0.0050      0.004      1.366      0.172      -0.002       0.012
total_ads                                  0.0006   2.33e-05     24.487      0.000       0.001       0.001
most_ads_hour                              0.0009      0.000      4.168      0.000       0.000       0.001
treatment:total_ads                        0.0002   2.38e-05      9.235      0.000       0.000       0.000
treatment:most_ads_hour                   -0.0002      0.000     -0.763      0.445      -0.001       0.000
==============================================================================
Omnibus:                   620685.937   Durbin-Watson:                   1.993
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         28560591.619
Skew:                           5.609   Prob(JB):                         0.00
Kurtosis:                      35.244   Cond. No.                     3.59e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.59e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Interpretación del modelo interactivo:

* R²: El valor de R² se mantiene en 0.049, igual que en el modelo aditivo. Esto indica que no hay una mejora sustancial en la capacidad explicativa global del modelo al incluir las interacciones.
* Interacciones:
  * Algunas interacciones, como treatment:total_ads (coef = 0.0002, p < 0.001) y treatment:C(most_ads_day)[T.Tuesday] (coef = 0.0099, p = 0.01), son estadísticamente significativas. Esto sugiere que el efecto del tratamiento puede variar dependiendo del número de anuncios vistos y del día de la semana.
* Coeficiente principal de treatment:
  * El coeficiente de treatment por sí solo ya no es significativo (coef = 0.0013, p = 0.751), porque ahora el efecto del tratamiento depende de las interacciones con las otras variables.

### ¿Existen mejoras en la precision con respecto a los resultados del modelo aditivo?

Al incluir las interacciones entre el tratamiento y las variables (total_ads, most_ads_day, most_ads_hour), no se observa una mejora en la precisión global del modelo es decir, el R² se mantiene en 0.049, igual que en el modelo aditivo. Sin embargo, el coeficiente **treatment** no es estadísticamente significativo.